In [1]:
import pickle
from tensorflow.keras.models import load_model

# Guardar el modelo con Keras
model_path = "modelo_entrenado.keras"

# Serializar la ruta al modelo
with open("model_path.pkl", "wb") as f:
    pickle.dump(model_path, f)

# Para cargar el modelo desde el archivo pickle
with open("model_path.pkl", "rb") as f:
    loaded_model_path = pickle.load(f)

model = load_model(loaded_model_path)
